In [44]:
from support_module import ImageToNumDataset, NoMaskModel

In [45]:
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [46]:
import torch
from torchvision.transforms.v2 import ToDtype, Normalize, Compose, PILToTensor

transform = Compose([
    PILToTensor(),
    ToDtype(torch.float32, scale=True),
    Normalize((0.5,), (0.5,))
])

In [47]:
dataset = ImageToNumDataset("data/train_images", answers_file="data/train_answers.csv", transform=transform)

In [48]:
from torch.utils.data import DataLoader, random_split

train_dataset, validation_dataset = random_split(dataset, (0.8, 0.2))

train_dataloader = DataLoader(train_dataset, batch_size=56, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=56, shuffle=False)

In [49]:
model = NoMaskModel()
model = model.to(DEVICE)

In [50]:
from torch import nn, optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-9)

In [51]:
from ignite.metrics import Accuracy, Loss
from ignite.engine import create_supervised_trainer, create_supervised_evaluator

trainer = create_supervised_trainer(model, optimizer, criterion, device=DEVICE)
evaluator = create_supervised_evaluator(model, metrics={'accuracy': Accuracy(), 'nll': Loss(criterion)}, device=DEVICE)

In [52]:
# Сбор потерь и метрик для построения графиков
train_loss_values = []
validation_loss_values = []
validation_accuracy_values = []

In [53]:
import logging


logging.basicConfig(level=logging.INFO)

In [54]:
EPOCHS = 25

In [55]:
from ignite.engine import Events


@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    # Запуск оценки модели на обучающем наборе данных
    evaluator.run(train_dataloader)
    metrics = evaluator.state.metrics
    # Сбор и вывод средней точности и потерь на обучающем наборе
    train_loss_values.append(metrics['nll'])
    logging.info(
        f"Training Results - Epoch: {engine.state.epoch}  "
        f"Avg accuracy: {metrics['accuracy']:.2f} "
        f"Avg loss: {metrics['nll']:.2f}"
    )

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    logging.info(f"Starting validation on epoch {engine.state.epoch}")
    # Запуск оценки модели на валидационном наборе данных
    evaluator.run(validation_dataloader)
    metrics = evaluator.state.metrics
    # Сбор и вывод средней точности и потерь на валидационном наборе
    validation_loss_values.append(metrics['nll'])
    validation_accuracy_values.append(metrics['accuracy'])
    logging.info(
        f"Validation Results - Epoch: {engine.state.epoch}  "
        f"Avg accuracy: {metrics['accuracy']:.3f} "
        f"Avg loss: {metrics['nll']:.3f}"
    )
    logging.info(f"End of validation on epoch {engine.state.epoch}")
    torch.save(model.state_dict(), "models/model.pt")

In [56]:
trainer.run(train_dataloader, max_epochs=EPOCHS, epoch_length=20)

INFO:ignite.engine.engine.Engine:Engine run starting with max_epochs=25.
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
[E thread_pool.cpp:110] Exception in thread pool task: mutex lock failed: Invalid argument
ERROR:ignite.engine.engine.Engine:Engine run is terminating due to exception: 


KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

# Графики обучения
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss_values, label='Training Loss')
plt.plot(validation_loss_values, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(validation_accuracy_values, label='Validation Accuracy', color='red')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
model.eval()

plt.figure(figsize=(15,4))
# Смотрим результат обучения
for i in range(20):
    x, y_true = validation_dataset[i]
    y_pred = torch.argmax(model(transform(x.to(DEVICE))))
    plt.subplot(2, 10, i + 1)
    plt.imshow(x.permute(1, 2, 0) * 0.25 + 0.5)
    plt.xticks([])
    plt.yticks([])
    class_to_idx = {value: key for key, value in validation_dataset.class_to_idx.items()}
    plt.title(f'True = {class_to_idx[y_true]}\nPred = {class_to_idx[y_pred.item()]}')

plt.show()

In [57]:
test_model = NoMaskModel()
test_model = test_model.to(DEVICE)
test_model.load_state_dict(torch.load("models/model.pt"))
test_dataset = ImageToNumDataset("data/test_images", transform=transform)

In [67]:
import csv
from IPython.display import clear_output

len_dataset = len(test_dataset)
with open("answer.csv", "w") as file:
    writer = csv.writer(file, delimiter=",")
    writer.writerow(["id", "target_feature"])
    for index, image in enumerate(test_dataset):
        with torch.no_grad():
            pred_y = model(image.unsqueeze(0))
        answer = max(((n, i) for i, n in enumerate(pred_y[0])), key=lambda x: x[0])[1]
        writer.writerow([index, answer])
        if index % 10 == 0 or index % 10 == 9:
            print(f"{(index / len_dataset) * 100:.2f}%")
print("100%")

0.00%
0.13%
0.14%
0.27%
0.29%
0.42%
0.43%
0.56%
0.58%
0.71%
0.72%
0.85%
0.87%
1.00%
1.01%
1.14%
1.16%
1.29%
1.30%
1.43%
1.45%
1.58%
1.59%
1.72%
1.73%
1.86%
1.88%
2.01%
2.02%
2.15%
2.17%
2.30%
2.31%
2.44%
2.46%
2.59%
2.60%
2.73%
2.75%
2.88%
2.89%
3.02%
3.03%
3.16%
3.18%
3.31%
3.32%
3.45%
3.47%
3.60%
3.61%
3.74%
3.76%
3.89%
3.90%
4.03%
4.05%
4.18%
4.19%
4.32%
4.34%
4.47%
4.48%
4.61%
4.62%
4.75%
4.77%
4.90%
4.91%
5.04%
5.06%
5.19%
5.20%
5.33%
5.35%
5.48%
5.49%
5.62%
5.64%
5.77%
5.78%
5.91%
5.92%
6.05%
6.07%
6.20%
6.21%
6.34%
6.36%
6.49%
6.50%
6.63%
6.65%
6.78%
6.79%
6.92%
6.94%
7.07%
7.08%
7.21%
7.23%
7.36%
7.37%
7.50%
7.51%
7.64%
7.66%
7.79%
7.80%
7.93%
7.95%
8.08%
8.09%
8.22%
8.24%
8.37%
8.38%
8.51%
8.53%
8.66%
8.67%
8.80%
8.82%
8.95%
8.96%
9.09%
9.10%
9.23%
9.25%
9.38%
9.39%
9.52%
9.54%
9.67%
9.68%
9.81%
9.83%
9.96%
9.97%
10.10%
10.12%
10.25%
10.26%
10.39%
10.40%
10.53%
10.55%
10.68%
10.69%
10.82%
10.84%
10.97%
10.98%
11.11%
11.13%
11.26%
11.27%
11.40%
11.42%
11.55%
11.56%
11.69%
11.71